# Predict low high models
Model selection for predict lo hi


To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
#%pip install binance-connector==1.13.0
%pip install pandas
%pip install scikit-learn
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install scikeras
%pip install keras-tuner

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Feature engineering
Read data from bot's history, resample to equal intervals, create X, y

In [2]:
import os
import glob
import pandas as pd
from datetime import timedelta,date

def read_last_data(strategy, days=1, n=None):
    """ Read last last day from bot history """
    
    def read_last(symbol: str, name: str, days=1):
        data_dir=f"./../data/yandex-cloud/{strategy}/Xy"
        file_paths = sorted([f"{data_dir}/{f}" for f in os.listdir(data_dir) if f.endswith(f"{symbol}_{name}.csv")])[-days:]
        print(file_paths)
        return pd.concat([pd.read_csv(f, parse_dates=True, index_col="datetime") for f in file_paths])

    data=read_last("BTCUSDT", "data", days)
    X=read_last("BTCUSDT", "X", days)
    y=read_last("BTCUSDT", "y", days)
    if n: 
        data=data.tail(n)
        X=X.tail(n)
        y=y.tail(n)
    return data,X,y

#strategy="SimpleKerasStrategy"
strategy="LSTMStrategy"
bidask,X_bot,y_bot = read_last_data(strategy, days=1, n=10000)

['./../data/yandex-cloud/LSTMStrategy/Xy/2023-05-15_BTCUSDT_data.csv']
['./../data/yandex-cloud/LSTMStrategy/Xy/2023-05-15_BTCUSDT_X.csv']
['./../data/yandex-cloud/LSTMStrategy/Xy/2023-05-15_BTCUSDT_y.csv']


Resample data to use equal time intervals

In [3]:
import re
def get_targets(bidask, predict_window="10s"):
    """ 
    Calculate targets - bid/ask bounds in future prediction window
    """
    fut_min=bidask[["bid", "ask"]][::-1].rolling(predict_window).min()[::-1].rename(columns={"bid":"bid_min_fut", "ask":"ask_min_fut"})
    fut_max=bidask[["bid", "ask"]][::-1].rolling(predict_window).max()[::-1].rename(columns={"bid":"bid_max_fut", "ask":"ask_max_fut"})
    return pd.concat([fut_min, fut_max], axis = 1)
    
def resampled(X: pd.DataFrame, y: pd.DataFrame, time_interval) -> (pd.DataFrame, pd.DataFrame):
    """ Resample to make equal intervals time series """
    if int(re.sub(r'\D', "", time_interval)) == 0:
        return X, y
    Xy=pd.concat([X,y], axis=1)
    diffmap = [(c, "sum") for c in Xy.columns if c.endswith("diff")]
    timemap = [(c, "last") for c in Xy.columns if c.startswith("time") and not c.endswith("diff")]
    l2map = [(c, "last") for c in Xy.columns if c.startswith("l2_")]
    
    futmap = [(c, "last") for c in Xy.columns if c.endswith("_fut")]
    colmap = dict(diffmap + timemap + l2map + futmap + [("spread", "last")])
    resampled = Xy.resample(time_interval).agg(colmap).dropna()
    return resampled[X.columns], resampled[y.columns]

predict_window="10s"
resample_interval="0s"
y_raw = get_targets(bidask, predict_window)
#X,y = resampled(X_bot, y_raw, resample_interval)
X,y=X_bot,y_raw
#pd.concat([X,y], axis=1).dropna()


,time_hour,time_minute,time_second,time_day_of_week,time_diff,spread,bid_diff,bid_vol_diff,ask_diff,ask_vol_diff,l2_bucket_-10,l2_bucket_-9,l2_bucket_-8,l2_bucket_-7,l2_bucket_-6,l2_bucket_-5,l2_bucket_-4,l2_bucket_-3,l2_bucket_-2,l2_bucket_-1,l2_bucket_0,l2_bucket_1,l2_bucket_2,l2_bucket_3,l2_bucket_4,l2_bucket_5,l2_bucket_6,l2_bucket_7,l2_bucket_8,l2_bucket_9,bid_min_fut,ask_min_fut,bid_max_fut,ask_max_fut
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-15 02:31:44.793577,2,31,44,0,122237000.0,0.01,0.00,0.00000,0.00,0.18405,0.00088,0.0,0.0,0.0,0.0014,0.0,0.0,0.0,0.0,18.86277,19.05375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27168.53,27168.54,27168.78,27168.79
2023-05-15 02:31:44.906546,2,31,44,0,112969000.0,0.01,0.00,0.00000,0.00,1.42951,0.00088,0.0,0.0,0.0,0.0014,0.0,0.0,0.0,0.0,18.86277,19.05375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27168.53,27168.54,27168.78,27168.79
2023-05-15 02:31:45.019818,2,31,45,0,113272000.0,0.01,0.00,-0.00900,0.00,0.00000,0.00088,0.0,0.0,0.0,0.0014,0.0,0.0,0.0,0.0,18.86277,19.05375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27168.53,27168.54,27168.78,27168.79
2023-05-15 02:31:45.135074,2,31,45,0,115256000.0,0.15,-0.14,0.02169,0.00,0.00000,0.00088,0.0,0.0,0.0,0.0014,0.0,0.0,0.0,0.0,18.86277,19.05375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27168.53,27168.54,27168.64,27168.79
2023-05-15 02:31:45.246262,2,31,45,0,111188000.0,0.14,0.00,0.00000,-0.01,-5.20455,0.00088,0.0,0.0,0.0,0.0014,0.0,0.0,0.0,0.0,18.86277,19.05375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27168.53,27168.54,27168.64,27168.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-15 06:02:33.047705,6,2,33,0,119973000.0,0.01,0.00,0.00000,0.00,0.08000,0.00087,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,21.52959,14.42962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27335.81,27335.82,27335.81,27335.82
2023-05-15 06:02:33.166926,6,2,33,0,119221000.0,0.01,0.00,0.00000,0.00,0.16000,0.00087,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,21.52959,14.42962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27335.81,27335.82,27335.81,27335.82
2023-05-15 06:02:33.290797,6,2,33,0,123871000.0,0.01,0.00,0.00000,0.00,0.32000,0.00087,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,21.52959,14.42962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27335.81,27335.82,27335.81,27335.82


In [6]:
 def train_test_split(X,y):
    test_size=0.3
    test_index = int(len(X)*(1-test_size))
    X_train, y_train, X_test, y_test = X.iloc[:test_index], y.iloc[:test_index], X.iloc[test_index:], y.iloc[test_index:]


    # Train/test split    
    x_pipe = Pipeline(
        [("xrs", RobustScaler()),
         #("xss", StandardScaler()),     
         ("xmms", MinMaxScaler())])
    x_pipe.fit(X_train)

    y_pipe = Pipeline(
        [("yrs", RobustScaler()),
         #("yss",StandardScaler()),     
         ("ymms",MinMaxScaler())])
    y_pipe.fit(y_train)

    #x_scaler, y_scaler = MinMaxScaler().fit(X_train), MinMaxScaler().fit(y_train)
    X_train, y_train = x_pipe.transform(X_train), y_pipe.transform(y_train)
    X_test, y_test = x_pipe.transform(X_test), y_pipe.transform(y_test)
    print(f"Train size: {X_train.shape[0]}, test size: {X_test.shape[0]}")
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = train_test_split(X, y)

Train size: 70000, test size: 30000


## Create the model


In [ ]:
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scikeras.wrappers import KerasRegressor
from keras.layers import *
from keras.layers import LSTM
from sklearn.compose import ColumnTransformer, make_column_selector
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner


# def create_model_prod(X_train, y_train, window_size):
#     model = Sequential()
#     model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
#     model.add(Dropout(0.2))
#     model.add(LSTM(32))         
#     model.add(Dropout(0.2))
#     model.add(Dense(20, activation='relu'))
#     model.add(Dense(y_train.shape[1], activation='linear'))
#     #model.add(Dense(y_train.shape[1], activation='softmax'))
#     model.compile(optimizer='adam', loss='mae', metrics=['mse'])
#     return model  
   

def create_model(X_train, y_train, window_size):
    model = Sequential()
    model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(LSTM(32))         
    model.add(Dropout(0.2))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='linear'))
    #model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='mae', metrics=['mse'])
    return model


def plot_history(history):
    """ Plot history loss and metrics"""
    # summarize history for accuracy
    for metric_name in history.history:
        # validation metrics names will be calculated from related train metrics
        if metric_name.startswith("val_"): continue

        # Plot metric and related test (val_..) metric
        names=[metric_name, f"val_{metric_name}"]
        for name in names:
            plt.plot(history.history[name])
        # Captions and show the plot
        plt.title(metric_name)
        plt.ylabel(metric_name)
        plt.xlabel('epoch')
        plt.legend(names, loc='upper left')
        plt.show()

def fit_model(model, train_gen, test_gen, window_size):
    # Fit the model
    epochs=2
    steps_per_epoch=5
    history=model.fit(train_gen, validation_data=test_gen, epochs=epochs, steps_per_epoch=steps_per_epoch)

    # Plot the result
    plot_history(history)    


def create_tuner_model(hp):
    """ Create model for tuner. Expects X_train, y_train, window_size"""
    model = Sequential()
#     model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
#     model.add(Dropout(0.2))
    hp_input_units=hp.Int("input_units", min_value=X_train.shape[1], max_value=1024, step=128)
    #hp_window_size=hp.Int("window_size", min_value=10, max_value=110, step=20)
    model.add(LSTM(hp_input_units,
                   return_sequences=True, input_shape=(10, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(LSTM(32))         
    model.add(Dropout(0.2))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='linear'))
    #model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='mae', metrics=['mse'])
    return model
    
window_size = 10
    
# Create the model
# model = create_model(X_train, y_train, window_size)
# fit_model(model, train_gen, test_gen, window_size)


# Tune the model
# tuner=keras_tuner.Hyperband(
#     hypermodel=create_tuner_model,
#     objective="val_mse",
#     max_epochs=5,
#     factor=3,
#     hyperband_iterations=1,
#     seed=None,
#     hyperparameters=None,
#     tune_new_entries=True,
#     allow_new_entries=True,
#     max_retries_per_trial=0,
#     max_consecutive_failed_trials=3
# )
tuner=keras_tuner.BayesianOptimization(
    hypermodel=create_tuner_model,
    objective="val_mse",
    max_trials=10,
    num_initial_points=None,
    alpha=0.0001,
    beta=2.6,
    seed=None,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    max_retries_per_trial=0,
    max_consecutive_failed_trials=3,
    overwrite=True,
    directory=f"./tmp/{strategy}/tune",
    project_name="biml",    
)
# tuner = keras_tuner.RandomSearch(
#     hypermodel=create_tuner_model,
#     objective="val_mse",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory=f"./tmp/{strategy}/tune",
#     project_name="biml",
#     )


tuner.search_space_summary()
tuner.search(train_gen,epochs=5, validation_data=test_gen)
models = tuner.get_best_models(num_models=3)
print("Best models:")
for m in models:
    m.summary()

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="my_dir",
#     project_name="helloworld",
# )



Trial 4 Complete [00h 00m 31s]
val_mse: 0.41865667700767517

Best val_mse So Far: 0.29278430342674255
Total elapsed time: 00h 04m 40s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
926               |30                |input_units

Epoch 1/5
547/547 [==============================] - 67s 118ms/step - loss: 0.1978 - mse: 0.0692 - val_loss: 0.6799 - val_mse: 0.5545
Epoch 2/5
547/547 [==============================] - 64s 117ms/step - loss: 0.1737 - mse: 0.0525 - val_loss: 0.6414 - val_mse: 0.5042
Epoch 3/5
547/547 [==============================] - 64s 117ms/step - loss: 0.1474 - mse: 0.0409 - val_loss: 0.6990 - val_mse: 0.5838
Epoch 4/5
547/547 [==============================] - 65s 118ms/step - loss: 0.1397 - mse: 0.0381 - val_loss: 0.6691 - val_mse: 0.5478
Epoch 5/5
 40/547 [=>............................] - ETA: 53s - loss: 0.1613 - mse: 0.0457